In [1]:
%matplotlib inline
import os
import re
import io
import itertools
import pprint

from bokeh.io import output_notebook, show
from bokeh.layouts import gridplot
from bokeh.plotting import figure
from bokeh.models import Range1d, Legend
from bokeh.palettes import all_palettes

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr

import climatools.lblnew.bestfit_params as bestfits
from climatools.lblnew import setup_bestfit, setup_overlap
import climatools.lblnew.pipeline as pipe_lblnew
import climatools.cliradlw.setup as setup_cliradlw
import climatools.cliradlw.pipeline as pipe_cliradlw

import climatools.html.html as climahtml
from climatools.lblnew.dataio import *
from climatools.plot.plot import *


import IPython.display as display

In [2]:
%run param.py

In [3]:
def load_output_file(path_csv):
    '''
    Load lblnew output .csv file to xarray.Dataset
    
    Parameters
    ----------
    path_csv: str
              Path to the .csv file to be loaded.
    ds: xarray.Dataset
        Data in the input file in the form of an xarray.Dataset.
    '''
    toindex = ['i', 'band', 'pressure', 'igg', 'g']    
    df = pd.read_csv(path_csv, sep=r'\s+')
    df = df.set_index([i for i in toindex if i in df.columns])
    df = df.rename(columns={'sfu': 'flug',
                            'sfd': 'fldg',
                            'fnet': 'fnetg',
                            'coolr': 'coolrg'})
    ds = xr.Dataset.from_dataframe(df)

    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    return ds

In [4]:
def lblnew_setup(param=None):
    if 'ng_refs' in param:
        return {'setup': setup_bestfit,
                'fname_flux_crd': 'output_flux.dat',
                'fname_cool_crd': 'output_coolr.dat',
                'fname_flux_wgt': 'output_wfluxg.dat',
                'fname_cool_wgt': 'output_wcoolrg.dat'}
    else:
        return {'setup': setup_overlap,
                'fname_flux_crd': 'output_flux.dat',
                'fname_cool_crd': 'output_coolr.dat',
                'fname_flux_wgt': 'output_wflux.dat',
                'fname_cool_wgt': 'output_wcoolr.dat'}

    
def load_lblnew_data(param):
    
    fname_dsname = [('fname_flux_crd', 'ds_flux_crd'),
                    ('fname_cool_crd', 'ds_cool_crd'),
                    ('fname_flux_wgt', 'ds_flux_wgt'),
                    ('fname_cool_wgt', 'ds_cool_wgt')]
    
    d = lblnew_setup(param)
    dir_fortran = pipe_lblnew.get_dir_case(param, setup=d['setup'])
    
    data_dict = {}
    for fname, dsname in fname_dsname:
        fpath = os.path.join(dir_fortran, d[fname])
        data_dict[dsname] = load_output_file(fpath)
    return data_dict
    

    
    

In [5]:
d = load_lblnew_data(PARAM_LBLNEW)

DS_FLUX_CRD = d['ds_flux_crd']
DS_COOL_CRD = d['ds_cool_crd']
DS_FLUX_WGT = d['ds_flux_wgt']
DS_COOL_WGT = d['ds_cool_wgt']

In [6]:
DIR_FORTRAN = pipe_cliradlw.get_fortran_dir(PARAM, 
                                            setup=setup_cliradlw)

PATH_FLUX = os.path.join(DIR_FORTRAN, 'output_flux.dat')
PATH_COOL = os.path.join(DIR_FORTRAN, 'output_coolr.dat')

DS_FLUX = load_output_file(PATH_FLUX)
DS_COOL = load_output_file(PATH_COOL)

In [7]:
output_notebook()

Loading BokehJS ...

In [8]:
def fmt_cool(ds_in):
    ds = ds_in.copy(deep=True)
    if 'igg' in ds.dims:
        ds = ds.sel(igg=1)

    if 'g' in ds.dims:
        ds = ds.sum('g')
        
    if 'i' in ds.dims:                                                                                               
        ds = ds.sel(i=ds.dims['i']) 
            
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band')
        except ValueError:
            ds = ds.sum('band')
                
    return ds['coolrg']


def pltdata_cooling_1(ds_cool=None,
                      ds_cool_crd=None, ds_cool_wgt=None):
    


    colors = all_palettes['Set1'][4]
    data = [{'label': 'CLIRAD `coolr_bands`',
             'srs': fmt_cool(ds_cool),
             'line_dash': 'dashed', 'line_width': 5,
             'color': colors[0], 'alpha': .6},
            {'label': 'CRD',
             'srs': fmt_cool(ds_cool_crd),
             'line_dash': 'solid', 'line_width': 1.5,
             'marker': 'circle', 'marker_size': 5,
             'color': colors[2], 'alpha': 1,},
            {'label': 'WGT igg=10 wgt_flux=2',
             'srs': fmt_cool(ds_cool_wgt),
             'line_dash': 'solid', 'line_width': 3,
             'marker': 'square', 'marker_size': 5,
             'color': colors[3], 'alpha': .6}]
    return data


def nice_xlims(pltdata=None, prange=None):
    
    def get_slice(srs):
        return srs.sel(pressure=slice(*prange))
    
    srss = [d['srs'] for d in pltdata]
    vmin = min([get_slice(srs).min() for srs in srss])
    vmax = max([get_slice(srs).max() for srs in srss])
    dv = (vmax - vmin) * .01
    return float(vmin - dv), float(vmax + dv)
    

def plt_cooling_bokeh(pltdata=None):
    '''
    Plot a list of cooling rate profiles using bokeh.
    '''
    ymin = 1e-2 
    ymax = 1020 
        
    p1 = figure(title="Linear pressure scale", 
                plot_width=400)        
    xmin, xmax = nice_xlims(pltdata, prange=(50, 1050))
    for d in pltdata:
        if 'marker' in d:
            getattr(p1, d['marker'])(d['srs'].values, 
                    d['srs'].coords['pressure'].values,
                    color=d['color'], alpha=.7)
        p1.line(d['srs'].values, 
                d['srs'].coords['pressure'].values,
                color=d['color'], alpha=d['alpha'], 
                line_width=d['line_width'], line_dash=d['line_dash'])
    
    p1.y_range = Range1d(ymax, ymin)
    p1.yaxis.axis_label = 'pressure [mb]'   
    p1.x_range = Range1d(xmin, xmax)
    p1.xaxis.axis_label = 'cooling rate [K/day]'
            
    p2 = figure(title='Log pressure scale', y_axis_type='log',
                plot_width=560)
    xmin, xmax = nice_xlims(pltdata, prange=(.01, 200))
    
    rs = []
    for d in pltdata:
        rd = []
        if 'marker' in d:
            r_mark = getattr(p2, d['marker'])(d['srs'].values, 
                        d['srs'].coords['pressure'].values,
                        color=d['color'], alpha=.7)
            rd.append(r_mark)
        r_line = p2.line(d['srs'].values, 
                    d['srs'].coords['pressure'].values,
                    color=d['color'], alpha=d['alpha'], 
                    line_width=d['line_width'], line_dash=d['line_dash'])
        rd.append(r_line)
      
        rs.append(rd)
        
    p2.y_range = Range1d(ymax, ymin)  
    p2.yaxis.axis_label = 'pressure [mb]'
    
    p2.x_range = Range1d(xmin, xmax)
    p2.xaxis.axis_label = 'cooling rate [K/day]'
    
    items = [(d['label'], r) for r, d in zip(rs, pltdata)]
    legend = Legend(items=items, location=(10, -30))
    legend.label_text_font_size = '8pt'
    
    p2.add_layout(legend, 'right')  
    
    show(gridplot(p1, p2, ncols=2, plot_height=500))
    

def script_plt_cooling():
    pltdata = pltdata_cooling_1(
        ds_cool=DS_COOL,
        ds_cool_crd=DS_COOL_CRD,
        ds_cool_wgt=DS_COOL_WGT)
    plt_cooling_bokeh(pltdata=pltdata)
    print('''FIGURE. Cooling rate profiles.''')

In [9]:
def fmt_flux(ds_in, ilevels=None):
    ds = ds_in.copy(deep=True)
    
    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    if ilevels:
        ds = ds.isel(pressure=ilevels)

    if 'i' in ds.dims:                                                                                               
        ds = ds.sel(i=ds.dims['i'])
        
    if 'igg' in ds.dims:
        ds = ds.sel(igg=[1])
        ds = ds.squeeze('igg').drop('igg')
    
    if 'g' in ds.dims:
        ds = ds.sum('g')
            
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band').drop('band')
        except ValueError:
            ds = ds.sum('band')
                
    df = ds.to_dataframe()
    df = df.set_index('level', append=True)               
    return df


def tbdata_flux(ds_flux=None, ds_flux_crd=None, ds_flux_wgt=None):
    tropopauses = {'mls': 40, 'saw': 45, 'trp': 37}
    
    if PARAM['atmpro'] == PARAM['atmpro']:
        atmpro = PARAM['atmpro']
    else:
        atmpro = 'mls'
        
    ilevels = [0, tropopauses[atmpro], -1]
    
    benchmark = {'label': 'CRD',
                 'df': fmt_flux(ds_flux_crd, ilevels=ilevels)}
    
    tbs = [{'label': 'WGT igg=10 wgt_flux=1',
            'df': fmt_flux(ds_flux_wgt, ilevels=ilevels)},
           {'label': 'CLIRAD',
            'df': fmt_flux(ds_flux, ilevels=ilevels)}]
    
    data = {'benchmark': benchmark, 'others': tbs}
    return data


def tb_flux(tbdata=None):
    benchmark = tbdata['benchmark']
    others = tbdata['others']
    
    def show_tb(d=None):
        display.display(d['df'])
        print('Table.', 'Fluxes.', d['label'])
        
    def show_tbdiff(d0=None, d=None):
        df = d['df']
        df.index = d0['df'].index
        display.display(df - d0['df'])
        print('Table.', 'Fluxes.', 
              '({}) - ({})'.format(d['label'], d0['label']))
        
    show_tb(benchmark)
    for d in others:
        show_tb(d)
        
    for d in others:
        show_tbdiff(d=d, d0=benchmark)
        
        
def script_tb_flux():
    data = tbdata_flux(ds_flux=DS_FLUX,
                       ds_flux_crd=DS_FLUX_CRD,
                       ds_flux_wgt=DS_FLUX_WGT)
    tb_flux(tbdata=data)

In [10]:
def band_map():
    '''
    Maps spectral bands in lblnew to spectral bands in clirad.
    '''
    lblnew2clirad = {'1': '1',
            '2': '2',
            '3a': '3',
            '3b': '4',
            '3c': '5',
            '4': '6', 
            '5': '7',
            '6': '8',
            '7': '9', 
            '8': '10',
            '9': '11'}
    
    clirad2lblnew = {clirad: lblnew 
                     for lblnew, clirad in lblnew2clirad.items()}
    return clirad2lblnew


def gasbands():
    bmap = band_map()
    gases = list(PARAM['molecule'].keys())
    
    bands = [bmap[str(b)] for b in PARAM['band']]
    return list(itertools.product(gases, bands))


def print_bestfit_params():
    print('Best-fit values for each (gas, band)')
    print('------------------------------------')
    for gas, band in gasbands():
        print('o {} band{}'.format(gas, band))
        param = bestfits.kdist_params(molecule=gas, band=band)
        for name, value in sorted(param.items()):
            print(2 * ' ' + '{} = {}'.format(name, value))

        
def print_input_param():        
    for name, value in PARAM.items():
        print('{} = {}'.format(name, value))
        
        
def print_lblnew_param():
    for name, value in sorted(PARAM_LBLNEW.items()):
        print('{} = {}'.format(name, value))


In [11]:
'''
Cooling rate profile by layer
'''
def script_coolr_bylayer():
    '''
    Displays cooling rate of each layer
    '''
    keys = ['CRD', 'CLIRAD `coolr_bands`']
    dss = [DS_COOL_CRD, DS_COOL]
    das = [fmt_cool(ds) for ds in dss]
    
    dfs = [da.to_dataframe() for da in das]
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'coolr_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))



In [12]:
'''
Fluxes by layer
'''
def script_flux_bylayer():
    '''
    Display fluxes of each layer
    '''
    keys = ['CRD', 'CLIRAD']
    dss = [DS_FLUX_CRD, DS_FLUX]
    dfs = [fmt_flux(ds) for ds in dss]
    
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'fluxes_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))


In [13]:
def show_html(s):
    display.display(display.HTML(s))

    
def show_markdown(s):
    display.display(display.Markdown(s))

    
def script():
    
    title = '### CLIRAD-LW'
    s_input = 'Input Parameters'
    s_lblnew_param = 'LBLNEW case compaired against'
    s_bestfits = 'Best-fit Parameters'
    s_plt_cooling = 'Cooling Rate Profiles'
    s_tb_flux = 'Flux Comparison'
    s_coolr_bylayer = 'Cooling Rate by Layer'
    s_flux_bylayer = 'Fluxes by Layer'
    
    # title
    show_markdown('### CLIRAD-LW')
    
    # TOC
    show_markdown('### Table of Contents')
    show_html(climahtml.getHTML_hrefanchor(s_input))
    show_html(climahtml.getHTML_hrefanchor(s_lblnew_param))
    show_html(climahtml.getHTML_hrefanchor(s_plt_cooling))
    show_html(climahtml.getHTML_hrefanchor(s_tb_flux))
    show_html(climahtml.getHTML_hrefanchor(s_bestfits))
    show_html(climahtml.getHTML_hrefanchor(s_coolr_bylayer))
    show_html(climahtml.getHTML_hrefanchor(s_flux_bylayer))
    
    # Input parameters
    show_html(climahtml.getHTML_idanchor(s_input))
    show_markdown(climahtml.getMarkdown_sectitle(s_input))
    print_input_param()
    
    # LBLNEW parameters 
    show_html(climahtml.getHTML_idanchor(s_lblnew_param))
    show_markdown(climahtml.getMarkdown_sectitle(s_lblnew_param))
    print_lblnew_param()
    
    # Cooling rate profiles
    show_html(climahtml.getHTML_idanchor(s_plt_cooling))
    show_markdown(climahtml.getMarkdown_sectitle(s_plt_cooling))
    script_plt_cooling()
    
    # Flux comparison
    show_html(climahtml.getHTML_idanchor(s_tb_flux))
    show_markdown(climahtml.getMarkdown_sectitle(s_tb_flux))
    script_tb_flux()
    
    # Best-fit values
    show_html(climahtml.getHTML_idanchor(s_bestfits))
    show_markdown(climahtml.getMarkdown_sectitle(s_bestfits))
    print_bestfit_params()
    
    # Cooling rate by layer
    show_html(climahtml.getHTML_idanchor(s_coolr_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_coolr_bylayer))
    script_coolr_bylayer()
    
    # Fluxes by layer
    show_html(climahtml.getHTML_idanchor(s_flux_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_flux_bylayer))
    script_flux_bylayer()
   
    

In [14]:
script()

### CLIRAD-LW

### Table of Contents

# Input Parameters

commitnumber = 523cbb7
molecule = {'co2': 0.0008}
band = [6]
atmpro = trp
tsfc = 300


# LBLNEW case compaired against

...

Best-fit values for each (gas, band)
------------------------------------
o co2 band4
  atmpro = mls
  band = 4
  commitnumber = a06b618
  conc = 0.0004
  dv = 0.001
  klin = 6.5e-24
  molecule = co2
  ng_adju = [0, 0]
  ng_refs = [1, 2]
  nv = 1000
  option_compute_btable = 0
  option_compute_ktable = 1
  option_wgt_flux = 1
  option_wgt_k = 1
  ref_pts = [(1, 250), (500, 250)]
  tsfc = 294
  vmax = 980
  vmin = 800
  w_diffuse = [(1.75,), (1.66, 1.9)]
  wgt = [(0.75,), (0.75, 0.95)]


# Cooling Rate by Layer

CRD           CLIRAD `coolr_bands`          
            layer    coolrg                layer    coolrg
pressure                                                  
0.000312        1 -0.001173                    1 -0.001206
0.000750        2 -0.001174                    2 -0.001231
0.001052        3 -0.001174                    3 -0.001203
0.001476        4 -0.001173                    4 -0.001208
0.002070        5 -0.001171                    5 -0.001210
0.002904        6 -0.001167                    6 -0.001213
0.004074        7 -0.001162                    7 -0.001212
0.005714        8 -0.001155                    8 -0.001211
0.008015        9 -0.001147                    9 -0.001208
0.011243       10 -0.001136                   10 -0.001207
0.015771       11 -0.001124                   11 -0.001214
0.022122       12 -0.001111                   12 -0.001215
0.031031       13 -0.001095                   13 -0.001216
0.043528       14 -0.001073                   14 -0.001213
0.061057       15 -0.000958                   15 -0.001168
0.085645       16 -0.000548                   16 -0.000952
0.120136       17  0.000371                   17 -0.000408
0.168516       18  0.001998                   18  0.000608
0.236378       19  0.004661                   19  0.002308
0.331549       20  0.008794                   20  0.004984
0.465100       21  0.014958                   21  0.009137
0.652400       22  0.022836                   22  0.014926
0.915100       23  0.025565                   23  0.017365
1.283650       24  0.021523                   24  0.014997
1.800600       25  0.015895                   25  0.011307
2.525700       26  0.010140                   26  0.007316
3.542800       27  0.005783                   27  0.004139
4.969550       28  0.002798                   28  0.001858
6.970850       29  0.000813                   29  0.000251
9.778100       30 -0.000464                   30 -0.000841
13.715850      31 -0.001248                   31 -0.001546
19.239350      32 -0.001694                   32 -0.001957
26.987250      33 -0.001911                   33 -0.002135
37.855300      34 -0.001974                   34 -0.002112
53.100050      35 -0.001870                   35 -0.001988
73.887500      36 -0.001585                   36 -0.001772
97.662500      37 -0.001407                   37 -0.001618
121.437500     38 -0.001618                   38 -0.001826
145.212500     39 -0.002060                   39 -0.002222
168.987500     40 -0.002448                   40 -0.002550
192.762500     41 -0.002758                   41 -0.002875
216.537500     42 -0.002970                   42 -0.003204
240.312500     43 -0.003069                   43 -0.003401
264.087500     44 -0.003047                   44 -0.003435
287.862500     45 -0.002896                   45 -0.003322
311.637500     46 -0.002611                   46 -0.003071
335.412500     47 -0.002186                   47 -0.002642
359.187500     48 -0.001619                   48 -0.002083
382.962500     49 -0.000907                   49 -0.001383
406.737500     50 -0.000049                   50 -0.000520
430.512500     51  0.000956                   51  0.000497
454.287500     52  0.002108                   52  0.001667
478.062500     53  0.003409                   53  0.003008
501.837500     54  0.004858                   54  0.004520
525.612500     55  0.006456                   55  0.006148
549.387500     56  0.008204                   56  0.007939
573.162500     57  0.010102                   57  0.009910
596.937500     58  0.012149                   58  0.012058
620.712500     59  0.014339                   59  0.014382
644.487500     60  0.016659                   60  0.016803
668.262500     61  0.019060                   61  0.019283
692.037500     62  0.021430                   62  0.021770
715.812500     63  0.023603                   63  0.024119
739.587500     64  0.025501                   64  0.026264
763.362500     65  0.027234                   65  0.028288
78

# Fluxes by Layer

CRD                              CLIRAD  \
                        flug          fldg      fnetg       flug   
pressure    level                                                  
0.000000    1     -66.435179  0.000000e+00 -66.435179 -66.469020   
0.000624    2     -66.435180  6.453487e-08 -66.435180 -66.469020   
0.000876    3     -66.435180  9.106681e-08 -66.435180 -66.469020   
0.001229    4     -66.435180  1.298663e-07 -66.435180 -66.469020   
0.001723    5     -66.435180  1.866558e-07 -66.435180 -66.469020   
0.002417    6     -66.435180  2.697407e-07 -66.435180 -66.469020   
0.003391    7     -66.435180  3.912921e-07 -66.435180 -66.469021   
0.004757    8     -66.435181  5.690974e-07 -66.435180 -66.469021   
0.006672    9     -66.435181  8.291405e-07 -66.435180 -66.469021   
0.009359    10    -66.435182  1.209411e-06 -66.435181 -66.469022   
0.013128    11    -66.435183  1.765408e-06 -66.435181 -66.469023   
0.018415    12    -66.435185  2.578237e-06 -66.435182 -66.469024   
0.025830    13    -66.435187  3.766370e-06 -66.435183 -66.469026   
0.036232    14    -66.435190  5.503122e-06 -66.435184 -66.469029   
0.050823    15    -66.435194  8.070235e-06 -66.435186 -66.469033   
0.071291    16    -66.435201  1.238880e-05 -66.435188 -66.469039   
0.100000    17    -66.435211  2.099367e-05 -66.435190 -66.469049   
0.140271    18    -66.435228  3.896097e-05 -66.435189 -66.469064   
0.196760    19    -66.435251  7.604212e-05 -66.435175 -66.469087   
0.275997    20    -66.435283  1.515497e-04 -66.435131 -66.469120   
0.387100    21    -66.435319  3.029619e-04 -66.435016 -66.469163   
0.543100    22    -66.435341  6.020017e-04 -66.434739 -66.469210   
0.761700    23    -66.435310  1.162132e-03 -66.434148 -66.469246   
1.068500    24    -66.435220  2.001083e-03 -66.433219 -66.469277   
1.498800    25    -66.435132  3.010198e-03 -66.432122 -66.469352   
2.102400    26    -66.435101  4.115667e-03 -66.430985 -66.469520   
2.949000    27    -66.435197  5.228618e-03 -66.429968 -66.469840   
4.136600    28    -66.435476  6.320753e-03 -66.429155 -66.470363   
5.802500    29    -66.435996  7.393183e-03 -66.428603 -66.471146   
8.139200    30    -66.436833  8.454911e-03 -66.428378 -66.472262   
11.417000   31    -66.438080  9.521721e-03 -66.428558 -66.473800   
16.014700   32    -66.439850  1.061231e-02 -66.429238 -66.475868   
22.464000   33    -66.442276  1.174423e-02 -66.430532 -66.478598   
31.510500   34    -66.445509  1.292863e-02 -66.432580 -66.482118   
44.200100   35    -66.449702  1.415524e-02 -66.435547 -66.486497   
62.000000   36    -66.454735  1.524508e-02 -66.439489 -66.491772   
85.775000   37    -66.459985  1.603241e-02 -66.443953 -66.497619   
109.550000  38    -66.464512  1.659563e-02 -66.447917 -66.502810   
133.325000  39    -66.469891  1.741647e-02 -66.452475 -66.508814   
157.100000  40    -66.477274  1.899820e-02 -66.458276 -66.516574   
180.875000  41    -66.486803  2.163144e-02 -66.465171 -66.526098   
204.650000  42    -66.498552  2.561494e-02 -66.472937 -66.537682   
228.425000  43    -66.512555  3.125349e-02 -66.481301 -66.551748   
252.200000  44    -66.528793  3.884833e-02 -66.489945 -66.568229   
275.975000  45    -66.547227  4.870039e-02 -66.498526 -66.586910   
299.750000  46    -66.567789  6.110642e-02 -66.506683 -66.607655   
323.525000  47    -66.590392  7.635680e-02 -66.514035 -66.630399   
347.300000  48    -66.614925  9.473411e-02 -66.520191 -66.654754   
371.075000  49    -66.641263  1.165125e-01 -66.524751 -66.680721   
394.850000  50    -66.669259  1.419524e-01 -66.527306 -66.708306   
418.625000  51    -66.698745  1.713006e-01 -66.527445 -66.737261   
442.400000  52    -66.729554  2.048009e-01 -66.524753 -66.767246   
466.175000  53    -66.761504  2.426876e-01 -66.518816 -66.798233   
489.950000  54    -66.794401  2.851852e-01 -66.509216 -66.830129   
513.725000  55    -66.828043  3.325089e-01 -66.495534 -66.862798   
537.500000  56    -66.862216  3.848655e-01 -66.477350 -66.895692   
561.275000  5

In [15]:
display.HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')